In [1]:
import praw
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import pandas as pd
import datetime as dt
import re 

## API

In [2]:
from pmaw import PushshiftAPI
api = PushshiftAPI()

## select subreddit

In [3]:
subreddit="environment"
limit=1000



# adding timeframe 
before = int(dt.datetime(2022,1,2,0,0).timestamp())
after = int(dt.datetime(2022,1,1,0,0).timestamp())

comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
print(f'Retrieved {len(comments)} comments from Pushshift {subreddit}')

Retrieved 763 comments from Pushshift environment


## Start scraping
Scraping the above selected subreddit for every day. Then saving it in a csv for per year.

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
temp_df = pd.DataFrame() #Temporary empty dataframe

year = 2015

# YEARLY SCRAPE
# If year has comments < 1000 scrape everything at once 
scrapeYearDateBefore = int(dt.datetime(year,12,31,0,0).timestamp())
scrapeYearDateAfter = int(dt.datetime(year,1,1,0,0).timestamp())

# Scrape all comments per Year
commentsYear = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeYearDateBefore, after=scrapeYearDateAfter)


if (len(commentsYear) > 0 and len(commentsYear) < 1000): 
    print(f'Retrieved {len(commentsYear)} comments in {year} from Pushshift')
    # convert comments into a datafrom
    comments_df = pd.DataFrame(commentsYear)

    #add all datapoints to a temporary dataframe
    temp_df = temp_df.append(comments_df, ignore_index=True)
    
    
# MONTLY SCRAPE
else:

    print(f'more thann {len(commentsYear)} comments in {year} in sr: {subreddit}. Attempting monthyly scraping...')
    
    for m in range(1, 13):
        yearAfter = year
        monthBefore = m
        monthAfter = m
        day = 32
        minutes = 0
        endMonthDate = 31
        
        if(m == 1) or (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m == 10) or (m == 12):
            endMonthDate = 31
        elif(m == 2):
            endMonthDate = 28
        else: 
            endMonthDate = 30
            
        # If month has comments < 1000 scrape all monthly data at once 
        scrapeMonthDateBefore = int(dt.datetime(year,m,endMonthDate,0,0).timestamp())
        scrapeMonthDateAfter = int(dt.datetime(year,m,1,0,0).timestamp())
        
        # Scrape all comments per month
        commentsMonth = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMonthDateBefore, after=scrapeMonthDateAfter)
        
        if (len(commentsMonth) > 0 and len(commentsMonth) < 1000): 
            print(f'Retrieved {len(commentsMonth)} comments in month: {m} - from Pushshift')
            # convert comments into a datafrom
            comments_df = pd.DataFrame(commentsMonth)

            #add all datapoints to a temporary dataframe
            temp_df = temp_df.append(comments_df, ignore_index=True)

    
    
    
        # DAILY SCRAPE
        else:
            print(f'more than {len(commentsMonth)} comments in month {m} - {year}')
            if (m == 2): #februari 
                day = 29
            if (m == 4) or (m == 6) or (m == 9) or (m == 11): # months with 30 days -> its days +1 for the loop
                day = 31

            for d in range(1, day):
                # date variables
                dayBefore = d
                dayAfter = d-1

                # change dayAfter / monthAfter on certain conditions
                if(dayAfter == 0): # 
                    monthAfter = m-1
                    if(m == 2):
                        dayAfter = 31
                    elif(m==3):
                        dayAfter = 28
                    elif (m == 4) or (m == 6) or (m == 9) or (m == 11) or (m == 13):
                        dayAfter = 31
                    else: dayAfter = 30
                else: monthAfter = m

                if(monthAfter == 0):
                    continue

                #set time frame for scraping -> Scraping for every day in the year.
                scrapeDayDateBefore = int(dt.datetime(year,monthBefore,dayBefore,0,0).timestamp())
                scrapeDayDateAfter = int(dt.datetime(yearAfter,monthAfter,dayAfter,0,0).timestamp())


                # Scrape all comments per day
                commentsDay = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeDayDateBefore, after=scrapeDayDateAfter)
                  
                # if statement day comments < 1000
                if (len(commentsDay) < 1000): 
                    print('datebefore', dayBefore, '-', monthBefore, '- ', year, 'date after', dayAfter, '-', monthAfter, '- ', yearAfter)
                    print(f'Retrieved {len(commentsDay)} comments per day from Pushshift')
                    # convert comments into a datafrom
                    comments_df = pd.DataFrame(commentsDay)

                    #add all datapoints to a temporary dataframe
                    temp_df = temp_df.append(comments_df, ignore_index=True)
      
    
    
    
    
                # HOURLY SCRAPE
                else: 
                    print(f'more than {len(commentsDay)} comments: failed daily scrape, attempting hourly scrape...')

                    for h in range(24):

                        # add an hour to the datetime object
                        hourBefore = h + 1
                        hourAfter = h
                        minuteAfter = 0

                        if (hourBefore == 24):
                            hourBefore = 23
                            minutes = 59
                        else: 
                            hourBefore = h + 1
                            minutes = 0



                        #set time frame for scraping -> Scraping for every hour in the day
                        scrapeHourDateBefore = int(dt.datetime(year,m,d,hourBefore,minutes).timestamp())
                        scrapeHourDateAfter = int(dt.datetime(yearAfter,m,d,hourAfter,minuteAfter).timestamp())      

                        # Scrape all comments per hour
                        commentsHour = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeHourDateBefore, after=scrapeHourDateAfter)

                        # if statement hour comments < 1000
                        if (len(commentsHour) < 1000): 
                            print('datebefore', dt.datetime(year,m,d,hourBefore,minutes), 'date after', dt.datetime(yearAfter,m,d,hourAfter,minuteAfter)) 
                            print(f'Retrieved {len(commentsHour)} comments per hour from Pushshift')
                            # convert comments into a datafrom
                            comments_df = pd.DataFrame(commentsHour)

                            #add all datapoints to a temporary dataframe
                            temp_df = temp_df.append(comments_df, ignore_index=True)


                        # SCRAPING PER 20 MINUTES
                        else:
                            print(f'more than {len(commentsHour)} per hour: failed to do an hourly scrape, attempting 20 minute scrapes scrape...')

                            for q in range(3):

                                minuteBefore = 20
                                minuteAfter = 0

                                if (q == 0):
                                    minuteBefore = 20
                                    minuteAfter = 0

                                if (q == 1):
                                    minuteBefore = 40
                                    minuteAfter = 20

                                if (q == 2):
                                    minuteBefore = 59
                                    minuteAfter = 40


                                scrapeMinuteDateBefore = int(dt.datetime(year,m,d,h,minuteBefore).timestamp())
                                scrapeMinuteDateAfter = int(dt.datetime(year,m,d,h,minuteAfter).timestamp())  


                                # Scrape all comments per 20 mins
                                commentsMinute = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMinuteDateBefore, after=scrapeMinuteDateAfter)

                                # if statement hour comments < 1000
                                if (len(commentsMinute) < 1000): 
                                    print('dateafter', dt.datetime(year,m,d,h,minuteAfter), 'date after', dt.datetime(year,m,d,h,minuteBefore)) 
                                    print(f'Retrieved {len(commentsMinute)} comments per 20 minutes from Pushshift')
                                    # convert comments into a datafrom
                                    comments_df = pd.DataFrame(commentsMinute)

                                    #add all datapoints to a temporary dataframe
                                    temp_df = temp_df.append(comments_df, ignore_index=True)                                   



                                else:
                                # convert comments into a datafrom
                                    comments_df = pd.DataFrame(commentsMinute)

                                    #add all datapoints to a temporary dataframe
                                    temp_df = temp_df.append(comments_df, ignore_index=True)

                                    print('ERROR MORE THAN 1000 per 20 minutes, however, still going on. INCOMPLETE')
                                    print(f'more than {len(commentsMinute)} comments on day {d}-{m}-{year} between {h}:{minuteAfter} and {h}:{minuteBefore}')
        #                             raise ValueError("More than 1000 comments in this HOUR.")





                                

# all datapoints in temporary dataframe to one dataframe
df1 = pd.DataFrame(temp_df)
                                      
# create the filename with the variable name embedded
filename = f'{subreddit}_{year}.csv'

# save the dataframe to the file with the embedded variable name
df1.to_csv(filename, encoding='utf-8', header=True, index=False)


more thann 1000 comments in 2015 in sr: environment. Attempting monthyly scraping...
more than 1000 comments in month 1 - 2015
datebefore 2 - 1 -  2015 date after 1 - 1 -  2015
Retrieved 104 comments per day from Pushshift
datebefore 3 - 1 -  2015 date after 2 - 1 -  2015
Retrieved 131 comments per day from Pushshift
datebefore 4 - 1 -  2015 date after 3 - 1 -  2015
Retrieved 166 comments per day from Pushshift
datebefore 5 - 1 -  2015 date after 4 - 1 -  2015
Retrieved 169 comments per day from Pushshift
datebefore 6 - 1 -  2015 date after 5 - 1 -  2015
Retrieved 180 comments per day from Pushshift
datebefore 7 - 1 -  2015 date after 6 - 1 -  2015
Retrieved 295 comments per day from Pushshift
datebefore 8 - 1 -  2015 date after 7 - 1 -  2015
Retrieved 196 comments per day from Pushshift
datebefore 9 - 1 -  2015 date after 8 - 1 -  2015
Retrieved 246 comments per day from Pushshift
datebefore 10 - 1 -  2015 date after 9 - 1 -  2015
Retrieved 167 comments per day from Pushshift
datebefo

datebefore 26 - 3 -  2015 date after 25 - 3 -  2015
Retrieved 205 comments per day from Pushshift
datebefore 27 - 3 -  2015 date after 26 - 3 -  2015
Retrieved 144 comments per day from Pushshift
datebefore 28 - 3 -  2015 date after 27 - 3 -  2015
Retrieved 292 comments per day from Pushshift
datebefore 29 - 3 -  2015 date after 28 - 3 -  2015
Retrieved 204 comments per day from Pushshift
datebefore 30 - 3 -  2015 date after 29 - 3 -  2015
Retrieved 110 comments per day from Pushshift
datebefore 31 - 3 -  2015 date after 30 - 3 -  2015
Retrieved 192 comments per day from Pushshift
more than 1000 comments in month 4 - 2015
datebefore 1 - 4 -  2015 date after 31 - 3 -  2015
Retrieved 204 comments per day from Pushshift
datebefore 2 - 4 -  2015 date after 1 - 4 -  2015
Retrieved 240 comments per day from Pushshift
datebefore 3 - 4 -  2015 date after 2 - 4 -  2015
Retrieved 248 comments per day from Pushshift
datebefore 4 - 4 -  2015 date after 3 - 4 -  2015
Retrieved 181 comments per day 

datebefore 17 - 6 -  2015 date after 16 - 6 -  2015
Retrieved 208 comments per day from Pushshift
datebefore 18 - 6 -  2015 date after 17 - 6 -  2015
Retrieved 319 comments per day from Pushshift
datebefore 19 - 6 -  2015 date after 18 - 6 -  2015
Retrieved 223 comments per day from Pushshift
datebefore 20 - 6 -  2015 date after 19 - 6 -  2015
Retrieved 120 comments per day from Pushshift
datebefore 21 - 6 -  2015 date after 20 - 6 -  2015
Retrieved 127 comments per day from Pushshift
datebefore 22 - 6 -  2015 date after 21 - 6 -  2015
Retrieved 156 comments per day from Pushshift
datebefore 23 - 6 -  2015 date after 22 - 6 -  2015
Retrieved 130 comments per day from Pushshift
datebefore 24 - 6 -  2015 date after 23 - 6 -  2015
Retrieved 160 comments per day from Pushshift
datebefore 25 - 6 -  2015 date after 24 - 6 -  2015
Retrieved 198 comments per day from Pushshift
datebefore 26 - 6 -  2015 date after 25 - 6 -  2015
Retrieved 173 comments per day from Pushshift
datebefore 27 - 6 - 

datebefore 8 - 9 -  2015 date after 7 - 9 -  2015
Retrieved 97 comments per day from Pushshift
datebefore 9 - 9 -  2015 date after 8 - 9 -  2015
Retrieved 79 comments per day from Pushshift
datebefore 10 - 9 -  2015 date after 9 - 9 -  2015
Retrieved 150 comments per day from Pushshift
datebefore 11 - 9 -  2015 date after 10 - 9 -  2015
Retrieved 284 comments per day from Pushshift
datebefore 12 - 9 -  2015 date after 11 - 9 -  2015
Retrieved 185 comments per day from Pushshift
datebefore 13 - 9 -  2015 date after 12 - 9 -  2015
Retrieved 111 comments per day from Pushshift
datebefore 14 - 9 -  2015 date after 13 - 9 -  2015
Retrieved 138 comments per day from Pushshift
datebefore 15 - 9 -  2015 date after 14 - 9 -  2015
Retrieved 241 comments per day from Pushshift
datebefore 16 - 9 -  2015 date after 15 - 9 -  2015
Retrieved 166 comments per day from Pushshift
datebefore 17 - 9 -  2015 date after 16 - 9 -  2015
Retrieved 170 comments per day from Pushshift
datebefore 18 - 9 -  2015 d

datebefore 30 - 11 -  2015 date after 29 - 11 -  2015
Retrieved 129 comments per day from Pushshift
more than 1000 comments in month 12 - 2015
datebefore 1 - 12 -  2015 date after 30 - 11 -  2015
Retrieved 182 comments per day from Pushshift
datebefore 2 - 12 -  2015 date after 1 - 12 -  2015
Retrieved 222 comments per day from Pushshift
datebefore 3 - 12 -  2015 date after 2 - 12 -  2015
Retrieved 224 comments per day from Pushshift
datebefore 4 - 12 -  2015 date after 3 - 12 -  2015
Retrieved 222 comments per day from Pushshift
datebefore 5 - 12 -  2015 date after 4 - 12 -  2015
Retrieved 169 comments per day from Pushshift
datebefore 6 - 12 -  2015 date after 5 - 12 -  2015
Retrieved 183 comments per day from Pushshift
datebefore 7 - 12 -  2015 date after 6 - 12 -  2015
Retrieved 211 comments per day from Pushshift
datebefore 8 - 12 -  2015 date after 7 - 12 -  2015
Retrieved 180 comments per day from Pushshift
datebefore 9 - 12 -  2015 date after 8 - 12 -  2015
Retrieved 235 commen

In [ ]:
print(len(df1))

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
temp_df = pd.DataFrame() #Temporary empty dataframe

year = 2016

# YEARLY SCRAPE
# If year has comments < 1000 scrape everything at once 
scrapeYearDateBefore = int(dt.datetime(year,12,31,0,0).timestamp())
scrapeYearDateAfter = int(dt.datetime(year,1,1,0,0).timestamp())

# Scrape all comments per Year
commentsYear = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeYearDateBefore, after=scrapeYearDateAfter)


if (len(commentsYear) > 0 and len(commentsYear) < 1000): 
    print(f'Retrieved {len(commentsYear)} comments in {year} from Pushshift')
    # convert comments into a datafrom
    comments_df = pd.DataFrame(commentsYear)

    #add all datapoints to a temporary dataframe
    temp_df = temp_df.append(comments_df, ignore_index=True)
    
    
# MONTLY SCRAPE
else:

    print(f'more thann {len(commentsYear)} comments in {year} in sr: {subreddit}. Attempting monthyly scraping...')
    
    for m in range(1, 13):
        yearAfter = year
        monthBefore = m
        monthAfter = m
        day = 32
        minutes = 0
        endMonthDate = 31
        
        if(m == 1) or (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m == 10) or (m == 12):
            endMonthDate = 31
        elif(m == 2):
            endMonthDate = 28
        else: 
            endMonthDate = 30
            
        # If month has comments < 1000 scrape all monthly data at once 
        scrapeMonthDateBefore = int(dt.datetime(year,m,endMonthDate,0,0).timestamp())
        scrapeMonthDateAfter = int(dt.datetime(year,m,1,0,0).timestamp())
        
        # Scrape all comments per month
        commentsMonth = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMonthDateBefore, after=scrapeMonthDateAfter)
        
        if (len(commentsMonth) > 0 and len(commentsMonth) < 1000): 
            print(f'Retrieved {len(commentsMonth)} comments in month: {m} - from Pushshift')
            # convert comments into a datafrom
            comments_df = pd.DataFrame(commentsMonth)

            #add all datapoints to a temporary dataframe
            temp_df = temp_df.append(comments_df, ignore_index=True)

    
    
    
        # DAILY SCRAPE
        else:
            print(f'more than {len(commentsMonth)} comments in month {m} - {year}')
            if (m == 2): #februari 
                day = 29
            if (m == 4) or (m == 6) or (m == 9) or (m == 11): # months with 30 days -> its days +1 for the loop
                day = 31

            for d in range(1, day):
                # date variables
                dayBefore = d
                dayAfter = d-1

                # change dayAfter / monthAfter on certain conditions
                if(dayAfter == 0): # 
                    monthAfter = m-1
                    if(m == 2):
                        dayAfter = 31
                    elif(m==3):
                        dayAfter = 28
                    elif (m == 4) or (m == 6) or (m == 9) or (m == 11) or (m == 13):
                        dayAfter = 31
                    else: dayAfter = 30
                else: monthAfter = m

                if(monthAfter == 0):
                    continue

                #set time frame for scraping -> Scraping for every day in the year.
                scrapeDayDateBefore = int(dt.datetime(year,monthBefore,dayBefore,0,0).timestamp())
                scrapeDayDateAfter = int(dt.datetime(yearAfter,monthAfter,dayAfter,0,0).timestamp())


                # Scrape all comments per day
                commentsDay = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeDayDateBefore, after=scrapeDayDateAfter)
                  
                # if statement day comments < 1000
                if (len(commentsDay) < 1000): 
                    print('datebefore', dayBefore, '-', monthBefore, '- ', year, 'date after', dayAfter, '-', monthAfter, '- ', yearAfter)
                    print(f'Retrieved {len(commentsDay)} comments per day from Pushshift')
                    # convert comments into a datafrom
                    comments_df = pd.DataFrame(commentsDay)

                    #add all datapoints to a temporary dataframe
                    temp_df = temp_df.append(comments_df, ignore_index=True)
      
    
    
            
    
                # HOURLY SCRAPE
                else: 
                    print(f'more than {len(commentsDay)} comments: failed daily scrape, attempting hourly scrape...')

                    for h in range(24):

                        # add an hour to the datetime object
                        hourBefore = h + 1
                        hourAfter = h
                        minuteAfter = 0

                        if (hourBefore == 24):
                            hourBefore = 23
                            minutes = 59
                        else: 
                            hourBefore = h + 1
                            minutes = 0



                        #set time frame for scraping -> Scraping for every hour in the day
                        scrapeHourDateBefore = int(dt.datetime(year,m,d,hourBefore,minutes).timestamp())
                        scrapeHourDateAfter = int(dt.datetime(yearAfter,m,d,hourAfter,minuteAfter).timestamp())      

                        # Scrape all comments per hour
                        commentsHour = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeHourDateBefore, after=scrapeHourDateAfter)

                        # if statement hour comments < 1000
                        if (len(commentsHour) < 1000): 
                            print('datebefore', dt.datetime(year,m,d,hourBefore,minutes), 'date after', dt.datetime(yearAfter,m,d,hourAfter,minuteAfter)) 
                            print(f'Retrieved {len(commentsHour)} comments per hour from Pushshift')
                            # convert comments into a datafrom
                            comments_df = pd.DataFrame(commentsHour)

                            #add all datapoints to a temporary dataframe
                            temp_df = temp_df.append(comments_df, ignore_index=True)


                        # SCRAPING PER 20 MINUTES
                        else:
                            print(f'more than {len(commentsHour)} per hour: failed to do an hourly scrape, attempting 20 minute scrapes scrape...')

                            for q in range(3):

                                minuteBefore = 20
                                minuteAfter = 0

                                if (q == 0):
                                    minuteBefore = 20
                                    minuteAfter = 0

                                if (q == 1):
                                    minuteBefore = 40
                                    minuteAfter = 20

                                if (q == 2):
                                    minuteBefore = 59
                                    minuteAfter = 40


                                scrapeMinuteDateBefore = int(dt.datetime(year,m,d,h,minuteBefore).timestamp())
                                scrapeMinuteDateAfter = int(dt.datetime(year,m,d,h,minuteAfter).timestamp())  


                                # Scrape all comments per 20 mins
                                commentsMinute = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMinuteDateBefore, after=scrapeMinuteDateAfter)

                                # if statement hour comments < 1000
                                if (len(commentsMinute) < 1000): 
                                    print('dateafter', dt.datetime(year,m,d,h,minuteAfter), 'date after', dt.datetime(year,m,d,h,minuteBefore)) 
                                    print(f'Retrieved {len(commentsMinute)} comments per 20 minutes from Pushshift')
                                    # convert comments into a datafrom
                                    comments_df = pd.DataFrame(commentsMinute)

                                    #add all datapoints to a temporary dataframe
                                    temp_df = temp_df.append(comments_df, ignore_index=True)                                   



                                else:
                                # convert comments into a datafrom
                                    comments_df = pd.DataFrame(commentsMinute)

                                    #add all datapoints to a temporary dataframe
                                    temp_df = temp_df.append(comments_df, ignore_index=True)

                                    print('ERROR MORE THAN 1000 per 20 minutes, however, still going on. INCOMPLETE')
                                    print(f'more than {len(commentsMinute)} comments on day {d}-{m}-{year} between {h}:{minuteAfter} and {h}:{minuteBefore}')
        #                             raise ValueError("More than 1000 comments in this HOUR.")


                                
                                
                                
                                

# all datapoints in temporary dataframe to one dataframe
df1 = pd.DataFrame(temp_df)
                                      
# create the filename with the variable name embedded
filename = f'{subreddit}_{year}.csv'

# save the dataframe to the file with the embedded variable name
df1.to_csv(filename, encoding='utf-8', header=True, index=False)


more thann 1000 comments in 2016 in sr: environment. Attempting monthyly scraping...
more than 1000 comments in month 1 - 2016
datebefore 2 - 1 -  2016 date after 1 - 1 -  2016
Retrieved 99 comments per day from Pushshift
datebefore 3 - 1 -  2016 date after 2 - 1 -  2016
Retrieved 177 comments per day from Pushshift
datebefore 4 - 1 -  2016 date after 3 - 1 -  2016
Retrieved 133 comments per day from Pushshift
datebefore 5 - 1 -  2016 date after 4 - 1 -  2016
Retrieved 111 comments per day from Pushshift
datebefore 6 - 1 -  2016 date after 5 - 1 -  2016
Retrieved 97 comments per day from Pushshift
datebefore 7 - 1 -  2016 date after 6 - 1 -  2016
Retrieved 114 comments per day from Pushshift
datebefore 8 - 1 -  2016 date after 7 - 1 -  2016
Retrieved 172 comments per day from Pushshift
datebefore 9 - 1 -  2016 date after 8 - 1 -  2016
Retrieved 133 comments per day from Pushshift
datebefore 10 - 1 -  2016 date after 9 - 1 -  2016
Retrieved 238 comments per day from Pushshift
datebefore

datebefore 26 - 3 -  2016 date after 25 - 3 -  2016
Retrieved 191 comments per day from Pushshift
datebefore 27 - 3 -  2016 date after 26 - 3 -  2016
Retrieved 212 comments per day from Pushshift
datebefore 28 - 3 -  2016 date after 27 - 3 -  2016
Retrieved 149 comments per day from Pushshift
datebefore 29 - 3 -  2016 date after 28 - 3 -  2016
Retrieved 234 comments per day from Pushshift
datebefore 30 - 3 -  2016 date after 29 - 3 -  2016
Retrieved 206 comments per day from Pushshift
datebefore 31 - 3 -  2016 date after 30 - 3 -  2016
Retrieved 494 comments per day from Pushshift
more than 1000 comments in month 4 - 2016
datebefore 1 - 4 -  2016 date after 31 - 3 -  2016
Retrieved 261 comments per day from Pushshift
datebefore 2 - 4 -  2016 date after 1 - 4 -  2016
Retrieved 464 comments per day from Pushshift
datebefore 3 - 4 -  2016 date after 2 - 4 -  2016
Retrieved 136 comments per day from Pushshift
datebefore 4 - 4 -  2016 date after 3 - 4 -  2016
Retrieved 137 comments per day 

datebefore 18 - 6 -  2016 date after 17 - 6 -  2016
Retrieved 137 comments per day from Pushshift
datebefore 19 - 6 -  2016 date after 18 - 6 -  2016
Retrieved 88 comments per day from Pushshift
datebefore 20 - 6 -  2016 date after 19 - 6 -  2016
Retrieved 108 comments per day from Pushshift
datebefore 21 - 6 -  2016 date after 20 - 6 -  2016
Retrieved 64 comments per day from Pushshift
datebefore 22 - 6 -  2016 date after 21 - 6 -  2016
Retrieved 176 comments per day from Pushshift
datebefore 23 - 6 -  2016 date after 22 - 6 -  2016
Retrieved 107 comments per day from Pushshift
datebefore 24 - 6 -  2016 date after 23 - 6 -  2016
Retrieved 260 comments per day from Pushshift
datebefore 25 - 6 -  2016 date after 24 - 6 -  2016
Retrieved 353 comments per day from Pushshift
datebefore 26 - 6 -  2016 date after 25 - 6 -  2016
Retrieved 208 comments per day from Pushshift
datebefore 27 - 6 -  2016 date after 26 - 6 -  2016
Retrieved 125 comments per day from Pushshift
datebefore 28 - 6 -  2

datebefore 9 - 9 -  2016 date after 8 - 9 -  2016
Retrieved 107 comments per day from Pushshift
datebefore 10 - 9 -  2016 date after 9 - 9 -  2016
Retrieved 167 comments per day from Pushshift
datebefore 11 - 9 -  2016 date after 10 - 9 -  2016
Retrieved 219 comments per day from Pushshift
datebefore 12 - 9 -  2016 date after 11 - 9 -  2016
Retrieved 139 comments per day from Pushshift
datebefore 13 - 9 -  2016 date after 12 - 9 -  2016
Retrieved 92 comments per day from Pushshift
datebefore 14 - 9 -  2016 date after 13 - 9 -  2016
Retrieved 139 comments per day from Pushshift
datebefore 15 - 9 -  2016 date after 14 - 9 -  2016
Retrieved 188 comments per day from Pushshift
datebefore 16 - 9 -  2016 date after 15 - 9 -  2016
Retrieved 130 comments per day from Pushshift
datebefore 17 - 9 -  2016 date after 16 - 9 -  2016
Retrieved 116 comments per day from Pushshift
datebefore 18 - 9 -  2016 date after 17 - 9 -  2016
Retrieved 149 comments per day from Pushshift
datebefore 19 - 9 -  201

datebefore 2016-11-10 19:00:00 date after 2016-11-10 18:00:00
Retrieved 15 comments per hour from Pushshift
datebefore 2016-11-10 20:00:00 date after 2016-11-10 19:00:00
Retrieved 22 comments per hour from Pushshift
datebefore 2016-11-10 21:00:00 date after 2016-11-10 20:00:00
Retrieved 27 comments per hour from Pushshift
datebefore 2016-11-10 22:00:00 date after 2016-11-10 21:00:00
Retrieved 24 comments per hour from Pushshift
datebefore 2016-11-10 23:00:00 date after 2016-11-10 22:00:00
Retrieved 21 comments per hour from Pushshift
datebefore 2016-11-10 23:59:00 date after 2016-11-10 23:00:00
Retrieved 14 comments per hour from Pushshift
datebefore 11 - 11 -  2016 date after 10 - 11 -  2016
Retrieved 831 comments per day from Pushshift
datebefore 12 - 11 -  2016 date after 11 - 11 -  2016
Retrieved 907 comments per day from Pushshift
more than 1000 comments: failed daily scrape, attempting hourly scrape...
datebefore 2016-11-13 01:00:00 date after 2016-11-13 00:00:00
Retrieved 95 com

datebefore 5 - 12 -  2016 date after 4 - 12 -  2016
Retrieved 311 comments per day from Pushshift
datebefore 6 - 12 -  2016 date after 5 - 12 -  2016
Retrieved 479 comments per day from Pushshift
datebefore 7 - 12 -  2016 date after 6 - 12 -  2016
Retrieved 344 comments per day from Pushshift
datebefore 8 - 12 -  2016 date after 7 - 12 -  2016
Retrieved 833 comments per day from Pushshift
datebefore 9 - 12 -  2016 date after 8 - 12 -  2016
Retrieved 368 comments per day from Pushshift
datebefore 10 - 12 -  2016 date after 9 - 12 -  2016
Retrieved 261 comments per day from Pushshift
datebefore 11 - 12 -  2016 date after 10 - 12 -  2016
Retrieved 300 comments per day from Pushshift
datebefore 12 - 12 -  2016 date after 11 - 12 -  2016
Retrieved 341 comments per day from Pushshift
datebefore 13 - 12 -  2016 date after 12 - 12 -  2016
Retrieved 329 comments per day from Pushshift
datebefore 14 - 12 -  2016 date after 13 - 12 -  2016
Retrieved 324 comments per day from Pushshift
datebefore 